In [1]:
import cv2
import numpy as np

video_path = "./input/o.mp4"
cap = cv2.VideoCapture(video_path)
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    resized_frame = cv2.resize(frame, (640, 320), interpolation=cv2.INTER_AREA)
    frames.append(resized_frame)
cap.release()

In [3]:
padded_frames = []
pad_w, pad_h = 16, 8
for frame in frames:
    h, w = frame.shape[:2]  # 320, 640
    # Add padding with horizontal wrapping
    padded = cv2.copyMakeBorder(
        frame,
        pad_h, pad_h, pad_w, pad_w,
        cv2.BORDER_WRAP  # Wraps horizontally
    )
    # Duplicate top row across top padding
    top_row = frame[0:1, :]  # Shape: (1, 640, 3)
    top_padding = np.tile(top_row, (pad_h, 1, 1))  # Repeat row 8 times: (8, 640, 3)
    # Extend width to 672 by padding or tiling (here, we pad with edge values)
    top_padding = cv2.copyMakeBorder(
        top_padding,
        0, 0, 0, pad_w * 2,  # Add 32 pixels total (16 each side)
        cv2.BORDER_REPLICATE  # Replicate edge pixels
    )  # Shape: (8, 672, 3)
    padded[:pad_h, :] = top_padding

    # Duplicate bottom row across bottom padding
    bottom_row = frame[-1:, :]  # Shape: (1, 640, 3)
    bottom_padding = np.tile(bottom_row, (pad_h, 1, 1))  # Shape: (8, 640, 3)
    bottom_padding = cv2.copyMakeBorder(
        bottom_padding,
        0, 0, 0, pad_w * 2,
        cv2.BORDER_REPLICATE
    )  # Shape: (8, 672, 3)
    padded[-pad_h:, :] = bottom_padding

    padded_frames.append(padded)

In [4]:
flow_magnitudes = []
for i in range(1, len(padded_frames)):
    prev = cv2.cvtColor(padded_frames[i-1], cv2.COLOR_BGR2GRAY)
    curr = cv2.cvtColor(padded_frames[i], cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(
        prev, curr, None,
        pyr_scale=0.3, levels=2, winsize=10, iterations=1, poly_n=5, poly_sigma=1.1, flags=0
    )
    magnitude = np.sqrt(flow[..., 0]**2 + flow[..., 1]**2)
    # Crop the padding (back to 640x320)
    cropped = magnitude[pad_h:-pad_h, pad_w:-pad_w]
    flow_magnitudes.append(cropped)

In [5]:
import csv

grid_data = []
for i in range(0, len(flow_magnitudes), 5):
    mag = flow_magnitudes[i]
    grid = np.zeros((64, 128))
    for y in range(64):
        for x in range(128):
            block = mag[y*5:(y+1)*5, x*5:(x+1)*5]
            grid[y, x] = np.mean(block)
    grid_data.append((i+1, grid.flatten()))  # Frame number (i+1 since flow starts at frame 1)

with open("optical_flow.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["frame_number"] + [f"grid_{i}" for i in range(128*64)])
    for frame_num, grid in grid_data:
        writer.writerow([frame_num] + grid.tolist())